# 1. trainer的完整流程

```
from transformers import Trainer, TrainerArguments

training_args = TrainingArguments(...)

trainer = Trainer(...)

trainer.train()

trainer.evaluate()

trainer.predict()
```

## 1.1 prepration

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import load_dataset
import torch
from torch.optim import Adam
import evaluate

checkpoint = "hfl/rbt3"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def process_function(example):
    inputs = tokenizer(example['review'], truncation=True, max_length=128)
    inputs["labels"] = example['label']
    return inputs
dataset = load_dataset('csv', data_files='data/ChnSentiCorp_htl_all.csv', split='train')
dataset = dataset.filter(lambda example: example['review'] is not None)
datasets = dataset.train_test_split(test_size=0.1)
tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets['train'].column_names)

# below are no longer needed
# collator = DataCollatorWithPadding(tokenizer=tokenizer)
# train_dataloader = DataLoader(tokenized_datasets['train'], collate_fn=collator, shuffle=True, batch_size=32)
# eval_dataloader = DataLoader(tokenized_datasets['test'], collate_fn=collator, shuffle=False, batch_size=64)    

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# below are no longer needed
# if torch.cuda.is_available():
    # model.cuda()
# optimizer = Adam(model.parameters(), lr=2e-5)  



Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 1.2 create evaluation function

In [2]:
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def eval_metrics(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')
    acc.update(f1)
    return acc

## 1.3 traininig arguments

In [6]:
# train_args = TrainingArguments(output_dir='./checkpoints', use_cpu=False, report_to='none')
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=50,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略 'epoch', 'steps' then add eval_steps = 100
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数 (模型个数)
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True,      # 训练完成后加载最优模型
                               report_to='none')    
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metrics)

## 1.4 rock and roll

In [7]:
# I encountered a bug here `RuntimeError: CUDA error: peer mapping resources exhausted`, so I have to use run them on a colab
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: CUDA error: peer mapping resources exhausted
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## 1.5 evaluation

In [15]:
# trainer.evaluate()
# trainer.evaluate(tokenized_datasets["test"])
# trainer.evaluate(tokenized_datasets["train"])

## 1.6 predict

In [17]:
# trainer.predict(tokenized_datasets["test"])